In [ ]:
parent_dir = "C:/Users/Davide/Desktop" #git/gitdf/EnergyCommunity.jl/run_cloud"

n_iter = 12
EC_size_list_enum = [10, 20]  # List of sizes of the EC to test in enum mode

In [ ]:
using EnergyCommunity
using FileIO
using HiGHS, Plots
using JuMP
using Gurobi
using Games
using TickTock
using Combinatorics
using DataFrames
using JLD2
using Latexify, LaTeXStrings
using YAML
using CSV

## Compare ENUM_MODE

In [ ]:
dict_enums = Dict(
    size_enum=>load("$parent_dir/results_paper/enum/enum_simulations_results_$size_enum.jld2")
    for size_enum in EC_size_list_enum
)

### ENUM_MODE: Create reward redistribution of enum modes

In [ ]:
df_reward_enum = DataFrame()

for EC_size in EC_size_list_enum
    df_reward_temp = deepcopy(dict_enums[EC_size]["df_reward_enum"])
    reward_list = setdiff(names(df_reward_temp), ["user_set"])
    rename!(df_reward_temp, reward_list .=> reward_list .* "_$EC_size")
    if nrow(df_reward_enum) == 0
        df_reward_enum = df_reward_temp
    else
        df_reward_enum = outerjoin(df_reward_enum, df_reward_temp, on=:user_set, makeunique=true)
    end
end

# sort by user set
sort!(df_reward_enum, :user_set)

df_reward_enum

### ENUM_MODE: Create comparison of computational time

In [ ]:
df_time_enum = DataFrame()

for EC_size in EC_size_list_enum
    df_time_temp = DataFrame(dict_enums[EC_size]["df_time_enum"])  # dict_time_enum

    df_time_temp[!, 2:end] ./= 3600  # change units to hours

    df_time_temp[!, "EC size"] = [EC_size]
    df_time_temp[!, "title"] = [L"Time [h]"]
    df_time_temp = df_time_temp[!, ["EC size"; "title"; names(df_time_temp)[1:end-2]]]

    if nrow(df_time_enum) == 0
        df_time_enum = df_time_temp
    else
        df_time_enum = vcat(df_time_enum, df_time_temp)
    end
end

df_time_enum

## Compare ITER_MODE

In [ ]:
df_run_simulations_iter = DataFrame([
    (EC_size=10, PoolSearchMode=1, PoolSolutions=10, precoal=[1], bestobjstop=true),
    (EC_size=10, PoolSearchMode=1, PoolSolutions=50, precoal=[1], bestobjstop=true),
    (EC_size=10, PoolSearchMode=1, PoolSolutions=200, precoal=[1], bestobjstop=true),
    (EC_size=10, PoolSearchMode=0, PoolSolutions=200, precoal=[1, 2], bestobjstop=true),
    (EC_size=10, PoolSearchMode=0, PoolSolutions=200, precoal=[1, 2], bestobjstop=false),
    (EC_size=10, PoolSearchMode=0, PoolSolutions=200, precoal=[1, 10], bestobjstop=true),
    (EC_size=10, PoolSearchMode=0, PoolSolutions=200, precoal=[1, 10], bestobjstop=false),
    (EC_size=10, PoolSearchMode=0, PoolSolutions=200, precoal=[1, 2, 3], bestobjstop=false),
    (EC_size=10, PoolSearchMode=0, PoolSolutions=200, precoal=[1, 9, 10], bestobjstop=false),
    (EC_size=20, PoolSearchMode=0, PoolSolutions=200, precoal=[1, 20], bestobjstop=false),
    (EC_size=50, PoolSearchMode=0, PoolSolutions=200, precoal=[1, 50], bestobjstop=false),
    (EC_size=100, PoolSearchMode=0, PoolSolutions=200, precoal=[1, 100], bestobjstop=false),
])

In [ ]:
dict_iter = Dict(
    id_run=>load("$parent_dir/results_paper/iter/iter_simulations_results_$id_run.jld2")
    for id_run in 1:n_iter
)

#### ITER_MODE: Comparison of reward distribution

In [ ]:
df_reward_iter = DataFrame()

for id_run in 1:n_iter
    df_reward_temp = deepcopy(dict_iter[id_run]["df_reward_iter"])
    reward_list = setdiff(names(df_reward_temp), ["user_set"])
    rename!(df_reward_temp, reward_list .=> reward_list .* "_$(nrow(df_reward_temp)-1)")
    if nrow(df_reward_iter) == 0
        df_reward_iter = df_reward_temp
    else
        df_reward_iter = outerjoin(df_reward_iter, df_reward_temp, on=:user_set, makeunique=true)
    end
end

# sort by user set
sort!(df_reward_iter, :user_set)
CSV.write("test.csv", df_reward_iter)
df_reward_iter

#### ITER_MODE: Create comparison of computational time

In [ ]:
df_time_iter = DataFrame()

for id_run in 1:n_iter
    df_time_temp = deepcopy(dict_iter[id_run]["df_time_iter"])

    df_time_temp[!, 3:end] ./= 3600  # change units to hours

    df_time_temp[!, "title"] = [L"Time [h]"]

    if nrow(df_time_iter) == 0
        df_time_iter = df_time_temp
    else
        df_time_iter = vcat(df_time_iter, df_time_temp)
    end
end

header_cols = ["name", names(df_run_simulations_iter)..., "title"]
df_list_rewards = setdiff(names(df_time_iter), ["id_run"; header_cols])

df_time_iter = hcat(df_run_simulations_iter[df_time_iter[!, "id_run"], :], df_time_iter[!, df_list_rewards])

CSV.write("comp_time.csv", df_time_iter)

df_time_iter

In [ ]:
df_list_rewards

In [ ]:
df_time_iter[!, [header_cols; df_list_rewards]]

In [ ]:
df_time_iter

In [ ]:
header_cols

In [ ]:
df_time_iter[!, setdiff(names(df_time_iter), ["id_run"; header_cols])]

In [ ]:
hcat(df_time_iter[!, setdiff(names(df_time_iter), ["id_run"; header_cols])], df_run_simulations_iter)

In [ ]:
hcat(df_time_iter, df_run_simulations_iter)